In [1]:
import numpy as np
import pandas as pd 
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
import os
import warnings
warnings.filterwarnings('ignore')
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import cross_val_score, GridSearchCV, RandomizedSearchCV
from sklearn.metrics import roc_curve,auc
from sklearn.preprocessing import MinMaxScaler, Imputer,StandardScaler,RobustScaler
from sklearn.linear_model import LogisticRegression,Ridge
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.preprocessing import PolynomialFeatures
from sklearn.ensemble import AdaBoostClassifier,RandomForestClassifier,BaggingClassifier,GradientBoostingClassifier
from sklearn.preprocessing import Imputer
from sklearn.model_selection import cross_val_score
import scipy as sp
import lightgbm as lgb
import random
%matplotlib inline
app_train = pd.read_csv('data/application_train.csv')
app_test = pd.read_csv('data/application_test.csv')

In [2]:
#X_train, X_test, y_train, y_test = train_test_split(app_train.drop('TARGET',axis = 1), app_train['TARGET'], test_size=0.30, random_state=425)

In [3]:
from sklearn.base import BaseEstimator, TransformerMixin

class TextSelector(BaseEstimator, TransformerMixin):
    def __init__(self,key):
        self.key = key
    def fit(self,X,y=None):
        return self
    def transform(self,X):
        return X[self.key]

class NumberSelector(BaseEstimator, TransformerMixin):
    def __init__(self,key):
        self.key=key
    def fit(self,X,y=None):
        return self
    def transform(self, X):
        return X[[self.key]]

In [4]:
#def preprocessing(df):
le = LabelEncoder()
le_count = 0
# if more than 2 values, use label encoding// else use one hot encoding
for col in app_train:
    if app_train[col].dtype == 'object':
        if len(list(app_train[col].unique())) <=2:
            le.fit(app_train[col])
            app_train[col] = le.transform(app_train[col])
            app_test[col] = le.transform(app_test[col])
            le_count +=1
print('%d columns were label encoded.' % le_count)   
app_train = pd.get_dummies(app_train) # one hot encoding
app_test = pd.get_dummies(app_test)
# align colums
train_labels = app_train['TARGET']
app_train, app_test = app_train.align(app_test,join='inner',axis = 1)
app_train['TARGET'] = train_labels

#add NaN for anomilies
app_train['DAYS_EMPLOYED_ANOM'] = app_train["DAYS_EMPLOYED"] == 365243
app_test["DAYS_EMPLOYED"].replace({365243: np.nan}, inplace = True)
app_train['DAYS_EMPLOYED'].replace({365243: np.nan}, inplace = True)
app_test['DAYS_EMPLOYED_ANOM'] = app_test["DAYS_EMPLOYED"] == 365243
app_train['DAYS_BIRTH'] = abs(app_train['DAYS_BIRTH'])

#build domain features
app_train_domain = app_train.copy()
app_test_domain = app_test.copy()
app_train_domain['CREDIT_INCOME_PERCENT'] = app_train_domain['AMT_CREDIT'] / app_train_domain['AMT_INCOME_TOTAL']
app_train_domain['ANNUITY_INCOME_PERCENT'] = app_train_domain['AMT_ANNUITY'] / app_train_domain['AMT_INCOME_TOTAL']
app_train_domain['CREDIT_TERM'] = app_train_domain['AMT_ANNUITY'] / app_train_domain['AMT_CREDIT']
app_train_domain['DAYS_EMPLOYED_PERCENT'] = app_train_domain['DAYS_EMPLOYED'] / app_train_domain['DAYS_BIRTH']
app_test_domain['CREDIT_INCOME_PERCENT'] = app_test_domain['AMT_CREDIT'] / app_test_domain['AMT_INCOME_TOTAL']
app_test_domain['ANNUITY_INCOME_PERCENT'] = app_test_domain['AMT_ANNUITY'] / app_test_domain['AMT_INCOME_TOTAL']
app_test_domain['CREDIT_TERM'] = app_test_domain['AMT_ANNUITY'] / app_test_domain['AMT_CREDIT']
app_test_domain['DAYS_EMPLOYED_PERCENT'] = app_test_domain['DAYS_EMPLOYED'] / app_test_domain['DAYS_BIRTH']

3 columns were label encoded.


In [5]:
#https://www.kaggle.com/willkoehrsen/intro-to-model-tuning-grid-and-random-search
model = GradientBoostingClassifier(max_depth=3,n_estimators=100)
#for model in models:
if 'TARGET' in app_train:
    train = app_train.drop(columns = ['TARGET'])
else:
    train = app_train.copy()
X=train
y=train_labels
#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=425)
pipeline = Pipeline([('imputer',Imputer(strategy='median')),('scaler',StandardScaler()),('clf', model)])
results = cross_val_score(pipeline, X, y, cv=5, n_jobs=-1, scoring='roc_auc')
print(results.min())

0.7457310054831172


In [6]:
pipeline.get_params

<bound method Pipeline.get_params of Pipeline(memory=None,
     steps=[('imputer', Imputer(axis=0, copy=True, missing_values='NaN', strategy='median', verbose=0)), ('scaler', StandardScaler(copy=True, with_mean=True, with_std=True)), ('clf', GradientBoostingClassifier(criterion='friedman_mse', init=None,
              learning_rate=0.1, loss='deviance', max_dept...      presort='auto', random_state=None, subsample=1.0, verbose=0,
              warm_start=False))])>

In [7]:
# Submission dataframe
submit = app_test[['SK_ID_CURR']]
submit['TARGET'] = pred
submit.to_csv('answer.csv',index=False)

NameError: name 'pred' is not defined

In [ ]:
#GradientBoostingClassifier()

In [ ]:
params = {'strategy':['mean','median']}

In [ ]:
#lgb.cv(random_params, train_set, num_boost_round = 10000, nfold = 3, early_stopping_rounds = 100, metrics = 'auc', seed = 42)

In [ ]:
param_dist = {
    'imputer__strategy': ['mean', 'median'],
    'clf__max_depth':[2,3],
    'clf__subsample':[0.33,0.66,1]
}

In [ ]:
pipeline.fit(X,y)
random_search = RandomizedSearchCV(pipeline, param_distributions=param_dist, n_iter=6, cv=3, scoring='roc_auc', n_jobs=-1, verbose=2)

In [42]:
random_search.fit(X,y)

Fitting 3 folds for each of 6 candidates, totalling 18 fits


[Parallel(n_jobs=-1)]: Done  13 out of  18 | elapsed: 39.4min remaining: 15.2min
[Parallel(n_jobs=-1)]: Done  18 out of  18 | elapsed: 42.1min finished


RandomizedSearchCV(cv=3, error_score='raise',
          estimator=Pipeline(memory=None,
     steps=[('imputer', Imputer(axis=0, copy=True, missing_values='NaN', strategy='median', verbose=0)), ('scaler', StandardScaler(copy=True, with_mean=True, with_std=True)), ('clf', GradientBoostingClassifier(criterion='friedman_mse', init=None,
              learning_rate=0.1, loss='deviance', max_dept...      presort='auto', random_state=None, subsample=1.0, verbose=0,
              warm_start=False))]),
          fit_params=None, iid=True, n_iter=6, n_jobs=-1,
          param_distributions={'imputer__strategy': ['mean', 'median'], 'clf__max_depth': [2, 3], 'clf__subsample': [0.33, 0.66, 1]},
          pre_dispatch='2*n_jobs', random_state=None, refit=True,
          return_train_score='warn', scoring='roc_auc', verbose=2)

In [43]:
random_search.best_score_

0.6996706630622581

In [44]:
random_search.best_params_

{'imputer__strategy': 'median', 'clf__subsample': 1, 'clf__max_depth': 2}